In [16]:
import os
import math
import pandas as pd
import concurrent.futures
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess
from glob import glob
import pybedtools
import re
import numpy as np
import matplotlib.ticker as ticker
from matplotlib.ticker import MultipleLocator
from scipy import stats
from statsmodels.stats.multitest import multipletests,fdrcorrection
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

#### 1. Allele pairing for protein-coding genes

Find one-to-one heterozygous allele pairs. Proteinortho takes in two faa files and outputs identified orthologs two-way. Here Pst104E hapA and hapB proteins were used. Synteny was used as additional info to discriminate orthologs.

`proteinortho -project=Puccinia_striiformis_Pst104E -cpus=26 -synteny Puccinia_striiformis_Pst104E_hapA.faa Puccinia_striiformis_Pst104E_hapB.faa`

Zhenyan Luo's [script](https://github.com/ZhenyanLuo/codes-used-for-mating-type/tree/main/4.%20Investigation%20of%20recombination%20suppression%20in%20MAT%20loci) was used to compute dS and dN values to identify divergent pairs (dS>0 or dN>0).

In [17]:
# load files
BASEDIR = "/media/ssd/rita/project/104e/allele_analysis"
proteinortho_file = os.path.join(BASEDIR, "allele_identification/output_analysed_alleles_test.df")
blast_fwd_out = os.path.join(BASEDIR, "reciprocal_blast_hit/A2B.out")
blast_rev_out = os.path.join(BASEDIR, "reciprocal_blast_hit/B2A.out")\

# import and clean up allele pairs and stats dataframe
proteinortho = pd.read_csv(proteinortho_file, sep="\t", usecols=range(1,16))

# filter allele pairs by reciprocal matches evalue < 0.05 both ways. this is for checking only, shouldn't change the df.
proteinortho = proteinortho[(proteinortho["evalue_ab"] < 0.05) & (proteinortho["evalue_ba"] < 0.05)]
proteinortho["Target"] = proteinortho["Target"].apply(lambda x: x[:-3])
proteinortho["Query"] = proteinortho["Query"].apply(lambda x: x[:-3])
proteinortho["Index"] = proteinortho["Index"].apply(lambda x: x.replace("-T1_", ",")[:-3])

# filter for one-to-one orthologs
proteinortho = proteinortho[~proteinortho["Target"].duplicated()]
proteinortho = proteinortho[~proteinortho["Query"].duplicated()]
het_proteinortho = proteinortho[(proteinortho["yn00_dS"] > 0) | (proteinortho["yn00_dN"] > 0)]
het_proteinortho

,Target,Query,evalue_ab,bitscore_ab,evalue_ba,bitscore_ba,same_strand,simscore,Index,protein_hamming,protein_levenshtein,cds_hamming,cds_levenshtein,yn00_dS,yn00_dN
15,Pst104E137_015700,Pst104E137_000068,0.000000e+00,2239.0,0.000000e+00,2239.0,1,1.000000,"Pst104E137_015700,Pst104E137_000068",0.0007,0.0007,0.0002,0.0002,0.0000,0.0003
19,Pst104E137_015721,Pst104E137_000089,8.780000e-79,289.0,1.120000e-78,289.0,1,1.000000,"Pst104E137_015721,Pst104E137_000089",0.2290,0.2290,0.2259,0.2259,0.0000,0.0030
20,Pst104E137_015723,Pst104E137_000092,3.600000e-41,163.0,3.570000e-41,163.0,1,0.606683,"Pst104E137_015723,Pst104E137_000092",0.0125,0.0125,0.0042,0.0042,0.0000,0.0062
26,Pst104E137_015745,Pst104E137_000113,4.270000e-67,250.0,4.240000e-67,250.0,1,0.995567,"Pst104E137_015745,Pst104E137_000113",0.0068,0.0068,0.0023,0.0023,0.0000,0.0033
27,Pst104E137_015746,Pst104E137_000114,1.140000e-39,158.0,1.130000e-39,158.0,1,0.584175,"Pst104E137_015746,Pst104E137_000114",0.0127,0.0127,0.0042,0.0042,0.0000,0.0056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10888,Pst104E137_023677,Pst104E137_008169,0.000000e+00,2506.0,0.000000e+00,2506.0,1,1.000000,"Pst104E137_023677,Pst104E137_008169",0.0000,0.0000,0.0003,0.0003,0.0008,0.0000
10889,Pst104E137_016604,Pst104E137_000990,0.000000e+00,2850.0,0.000000e+00,2850.0,1,1.000000,"Pst104E137_016604,Pst104E137_000990",0.0000,0.0000,0.0002,0.0002,0.0007,0.0000
10890,Pst104E137_029435,Pst104E137_013995,0.000000e+00,3120.0,0.000000e+00,3120.0,1,1.000000,"Pst104E137_029435,Pst104E137_013995",0.0000,0.0000,0.0002,0.0002,0.0007,0.0000
10891,Pst104E137_022568,Pst104E137_007023,0.000000e+00,2869.0,0.000000e+00,2869.0,1,1.000000,"Pst104E137_022568,Pst104E137_007023",0.2325,0.2325,0.2327,0.2327,0.0006,0.0000


#### 2. Differential allele-specific expression

For transcript abundance quantification see bambu_transcript_quant.R

Prepare transcript abundance matrix as DEseq2 input. Concatenate bambu abundance table for hapA and hapB horizontally. Named as condition_replicate_haplotype.

In [14]:
def get_gene_id(faa):
    """
    get gene ID from faa headers' second field
    """
    out = []
    with open(faa, "r") as file:
        for n in file.readlines():
            if n.startswith(">"):
                out.append(n.split(" ")[-1].rstrip())
    return out

In [22]:
BAMBU_GENECOUNTS_TSV = "allele_specific_expression/all_cond.gene_counts.cds.bambu.tsv"
HAPA_PROT = "allele_specific_expression/Puccinia_striiformis_Pst104E_hapA.proteins.fa"
HAPB_PROT = "allele_specific_expression/Puccinia_striiformis_Pst104E_hapB.proteins.fa"

a_gene_id = get_gene_id(HAPA_PROT)
b_gene_id = get_gene_id(HAPB_PROT)

allele_pair_dict = dict(zip(het_proteinortho["Query"], het_proteinortho["Target"]))
a_het_gene = allele_pair_dict.keys()
b_het_gene = allele_pair_dict.values()

# load bambu abundance tsv. note tRNA genes are not considered in this analysis
bambu_df = pd.read_csv(BAMBU_GENECOUNTS_TSV, sep="\t")
bambu_df.columns = [n.split("Pst104E_")[-1].split(".ont.mapped")[0] for n in bambu_df.columns]

a_bambu_df = bambu_df[bambu_df["GENEID"].isin(a_het_gene)]
b_bambu_df = bambu_df[bambu_df["GENEID"].isin(b_het_gene)]
a_bambu_df.columns = ["allele_a"]+list(a_bambu_df.columns)[1:]
b_bambu_df.columns = ["allele_b"]+list(b_bambu_df.columns)[1:]

merged_bambu_df = pd.DataFrame(list(allele_pair_dict.items()), columns=["allele_a", "allele_b"])
merged_bambu_df["allele_pair_id"] = merged_bambu_df["allele_a"]+[":"]+merged_bambu_df["allele_b"]
merged_bambu_df = pd.merge(merged_bambu_df, a_bambu_df, on="allele_a")
merged_bambu_df = pd.merge(merged_bambu_df, b_bambu_df, on="allele_b", suffixes=["_a", "_b"])
merged_bambu_df = merged_bambu_df.drop(columns=["allele_a", "allele_b"])

merged_bambu_df

,allele_pair_id,U_UG_rep1_a,U_UG_rep2_a,U_UG_rep3_a,U_UG_rep4_a,U_4dpi_rep1_a,U_4dpi_rep2_a,U_4dpi_rep3_a,U_4dpi_rep4_a,U_6dpi_rep1_a,...,U_8dpi_rep3_b,U_8dpi_rep4_b,U_10dpi_rep1_b,U_10dpi_rep2_b,U_10dpi_rep3_b,U_10dpi_rep4_b,U_12dpi_rep1_b,U_12dpi_rep2_b,U_12dpi_rep3_b,U_12dpi_rep4_b
0,Pst104E137_000068:Pst104E137_015700,78,58,76,85,0,6,0,1,1,...,41,30,53,55,62,59,141,72,61,177
1,Pst104E137_000089:Pst104E137_015721,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,Pst104E137_000092:Pst104E137_015723,12,14,9,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Pst104E137_000113:Pst104E137_015745,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Pst104E137_000114:Pst104E137_015746,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8117,Pst104E137_008169:Pst104E137_023677,76,55,83,94,0,3,4,0,0,...,33,12,60,52,64,90,98,59,59,133
8118,Pst104E137_000990:Pst104E137_016604,46,44,52,55,1,1,2,1,1,...,34,23,55,50,53,78,65,28,42,75
8119,Pst104E137_013995:Pst104E137_029435,176,156,153,193,4,0,0,3,6,...,94,56,107,81,139,132,145,77,99,226
8120,Pst104E137_007023:Pst104E137_022568,91,68,105,149,0,0,0,1,0,...,32,24,60,80,81,104,164,63,84,253


Keep rows that have at least 4 samples with transcript abundance greater than 5

In [26]:
condition = merged_bambu_df.iloc[:,1:].gt(5).sum(axis=1) >= 4
merged_bambu_df_filt = merged_bambu_df[condition]
merged_bambu_df_filt.to_csv("allele_specific_expression/allele_paired.gene_counts.cds.FILTERED.bambu.tsv", sep="\t", index=None)
merged_bambu_df_filt

,allele_pair_id,U_UG_rep1_a,U_UG_rep2_a,U_UG_rep3_a,U_UG_rep4_a,U_4dpi_rep1_a,U_4dpi_rep2_a,U_4dpi_rep3_a,U_4dpi_rep4_a,U_6dpi_rep1_a,...,U_8dpi_rep3_b,U_8dpi_rep4_b,U_10dpi_rep1_b,U_10dpi_rep2_b,U_10dpi_rep3_b,U_10dpi_rep4_b,U_12dpi_rep1_b,U_12dpi_rep2_b,U_12dpi_rep3_b,U_12dpi_rep4_b
0,Pst104E137_000068:Pst104E137_015700,78,58,76,85,0,6,0,1,1,...,41,30,53,55,62,59,141,72,61,177
2,Pst104E137_000092:Pst104E137_015723,12,14,9,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Pst104E137_000121:Pst104E137_015751,44,44,29,33,1,3,0,0,0,...,2,8,19,10,19,16,38,34,22,79
6,Pst104E137_000120:Pst104E137_015752,4,3,2,0,5,7,1,4,10,...,179,96,273,179,250,344,377,228,262,495
7,Pst104E137_000142:Pst104E137_015769,107,129,103,139,0,4,2,1,5,...,53,29,112,65,93,89,136,87,80,172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8117,Pst104E137_008169:Pst104E137_023677,76,55,83,94,0,3,4,0,0,...,33,12,60,52,64,90,98,59,59,133
8118,Pst104E137_000990:Pst104E137_016604,46,44,52,55,1,1,2,1,1,...,34,23,55,50,53,78,65,28,42,75
8119,Pst104E137_013995:Pst104E137_029435,176,156,153,193,4,0,0,3,6,...,94,56,107,81,139,132,145,77,99,226
8120,Pst104E137_007023:Pst104E137_022568,91,68,105,149,0,0,0,1,0,...,32,24,60,80,81,104,164,63,84,253
